In [1]:
import requests
from elasticsearch import Elasticsearch
import tiktoken
from openai import OpenAI

In [2]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

# Q2

In [4]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [5]:
index_name = "llm-zoomcamp-hw1"
es_client = Elasticsearch("http://localhost:9200")
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'llm-zoomcamp-hw1'})

In [6]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

# Q3

In [7]:
search_query = {
    "size": 10,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do I execute a command in a running docker container?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [8]:
response['hits']['hits'][0]['_score']

83.629654

# Q4

In [9]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do I execute a command in a running docker container?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [10]:
response['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

# Q5

In [11]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

In [12]:
for doc in response['hits']['hits']:
    context = context + context_template.format(question=doc['_source']['question'], text=doc['_source']['text']) +"\n\n"

In [13]:
prompt = prompt_template.format(question="How do I execute a command in a running docker container?", context=context).strip()

In [14]:
len(prompt)

1462

# Q6

In [15]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [16]:
len(encoding.encode(prompt))

322

# BONUS

In [18]:
client = OpenAI()
    
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content':prompt}]
)

print(response.choices[0].message.content)

To execute a command in a running Docker container, you can use the `docker exec` command. Here are the steps to do so:

1. First, find the container ID of the running container by using the `docker ps` command:
   ```sh
   docker ps
   ```

2. Once you have the container ID, you can execute a command in the specific container by using the `docker exec` command. For example, to start a bash session inside the container, you can run:
   ```sh
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID of the container you want to interact with.


In [20]:
price_per_request = 0.005*150/1000 + 0.015*250/1000
print(f'The price per request is ${price_per_request}')

The price per request is $0.0045


In [21]:
price_per_1000request = 0.005*150 + 0.015*250
print(f'The price per 1000 requests is ${price_per_1000request}')

The price per 1000 requests is $4.5
